In [13]:
import keras
keras.__version__

'2.4.3'

# **1) LSTM 으로 텍스트 생성하기**

## 1-1 ) 데이터 전처리 (니체의 손글씨 데이터)

In [14]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()  # 말뭉치를 소문자로 변환
print('말뭉치 크기:', len(text))

606208/600901 [==============================] - 1s 2us/step
말뭉치 크기: 600893


In [15]:
text[0:10]

'preface\n\n\n'

In [16]:
type(text)

str

In [17]:
maxlen = 60  # 시퀀스의 길이

step = 3  # 세 글자씩 건너 뛰면서 새로운 시퀀스를 샘플링
 
# 추출한 시퀀스를 담을 리스트
sentences = [] 

# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print(sentences[0])
print('\n')
print(sentences[1])  # p,r,e 이 세 글자를 건너뛴 것을 알 수 있음.
print('\n')
print('시퀀스 개수:', len(sentences))  # 약 60만 나누기 3

preface


supposing that truth is a woman--what then? is the


face


supposing that truth is a woman--what then? is there 


시퀀스 개수: 200278


In [18]:

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(text)))
print(chars)
print('고유한 글자:', len(chars))  # 60만개의 글자가, 57개의 글자로 조합됨.

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
고유한 글자: 57


In [19]:
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}


In [20]:
# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
print(len(sentences), maxlen, len(chars))

# 영행렬 생성
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
y

벡터화...
200278 60 57


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [21]:
first_sentence = sentences[:1]  # 한 문장당 60개의 글자
print(first_sentence)
for i, sentence in enumerate(first_sentence):

    for t, char in enumerate(sentence):
        print(i, t, char_indices[char])
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print(x[0,0])  # 0번째 문장, 0번째 글자의 one-hot encoding 결과
print(x[0,1])

['preface\n\n\nsupposing that truth is a woman--what then? is the']
0 0 42
0 1 44
0 2 31
0 3 32
0 4 27
0 5 29
0 6 31
0 7 0
0 8 0
0 9 0
0 10 45
0 11 47
0 12 42
0 13 42
0 14 41
0 15 45
0 16 35
0 17 40
0 18 33
0 19 1
0 20 46
0 21 34
0 22 27
0 23 46
0 24 1
0 25 46
0 26 44
0 27 47
0 28 46
0 29 34
0 30 1
0 31 35
0 32 45
0 33 1
0 34 27
0 35 1
0 36 49
0 37 41
0 38 39
0 39 27
0 40 40
0 41 8
0 42 8
0 43 49
0 44 34
0 45 27
0 46 46
0 47 1
0 48 46
0 49 34
0 50 31
0 51 40
0 52 23
0 53 1
0 54 35
0 55 45
0 56 1
0 57 46
0 58 34
0 59 31
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False False]
[False False False False False False False False False False False False
 False False False False False False False False False

## 1-2 ) 네트워크 구성 (LSTM 층과 Dense 분류기)

In [22]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))  # 128개값을 input으로 함
model.add(layers.Dense(len(chars), activation='softmax'))   # len(chars)=58 을 output 크기로 함

In [23]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## 1-3) 언어 모델 훈련과 샘플링

In [24]:
# 확률값을 계산해, 그 확률이 높은 글자를 샘플링
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [25]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 한 번의 에포크 동안만 훈련
for epoch in range(1, 2):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습합니다
    model.fit(x, y, batch_size=128, epochs=1)

    # 무작위로 시드 텍스트를 선택합니다
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 조정하면서 문장을 생성
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성합니다
        for i in range(400):
            # 지금까지 생성된 글자를 원-핫 인코딩으로 바꿉니다
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링합니다
            # softmax 함수를 통과한 결과값(58개)이 출력
            preds = model.predict(sampled, verbose=0)[0]
            print('\n 다음글자',preds)

            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            print('next_char : ', next_char)

            generated_text += next_char
            print('\n generated_text : ',generated_text)
            generated_text = generated_text[1:]

            #sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Streaming output truncated to the last 5000 lines.
 0.00276543 0.00597571 0.00318182 0.00604689 0.00119931 0.03459416
 0.01854633 0.01137735 0.00237784 0.00813003 0.00620391 0.00311571
 0.00927572 0.02269848 0.09186953 0.01040482 0.0112448  0.00651276
 0.00308619 0.00469039 0.01792094 0.00707212 0.00764585 0.03708694
 0.01664716 0.04814076 0.02065711 0.04542878 0.00742148 0.00609953
 0.07062823 0.00158534 0.00224773 0.03007427 0.00353617 0.02662482
 0.00166935 0.01505599 0.01507254]
next_char :  t

 generated_text :  gh fo.frev1väë)v631[-f-oqfrsv=-[(-n3,-k5ë-3)l3yq
tf3zo!4)h3ät

 다음글자 [0.00594286 0.00504795 0.00440774 0.07339542 0.00846301 0.02712289
 0.01585351 0.02068192 0.02810025 0.01494399 0.01196247 0.03343103
 0.00643047 0.04754409 0.00652456 0.01431546 0.02506009 0.00685439
 0.00276537 0.00597559 0.00318137 0.00604677 0.00119929 0.03458922
 0.01854595 0.01137711 0.0023778  0.00813185 0.00620378 0.00311527
 0.00927553 0.02269802 0.09189007 0.0104046  0.01124457 0.00651262
 0.003

에포크 수가 많아지거나 온도가 달라짐에 따라 다양한 pred 값이 나온다.